In [ ]:
# Imports Escenciales

import torch
import torchvision.transforms as tv_transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import sys
import copy


print("PyTorch version:", torch.__version__)
print("*"*20)
print("CUDA Version:")
!nvcc --version
print("*"*20)
print("cuDNN version:", torch.backends.cudnn.version())
print("Available GPUs:", torch.cuda.device_count())
print("CUDA available:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")


PyTorch version: 2.7.0+cu126
********************
CUDA Version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Wed_Apr__9_19:29:17_Pacific_Daylight_Time_2025
Cuda compilation tools, release 12.9, V12.9.41
Build cuda_12.9.r12.9/compiler.35813241_0
********************
cuDNN version: 90701
Available GPUs: 1
CUDA available: NVIDIA GeForce RTX 4050 Laptop GPU


In [26]:
#Imports de StyleGAN-NADA

current_working_directory = os.getcwd()
zssgan_repo_path = os.path.join(current_working_directory, "StyleGAN-nada/ZSSGAN")

if not os.path.isdir(zssgan_repo_path):
    print(f"ERROR: El directorio ZSSGAN no se encuentra en '{zssgan_repo_path}'.")
else:
    if zssgan_repo_path not in sys.path:
        sys.path.append(zssgan_repo_path)
        print(f"Ruta '{zssgan_repo_path}' añadida a sys.path.")
    else:
        dnnlib_dir_exists = os.path.isdir(os.path.join(zssgan_repo_path, 'dnnlib'))
        legacy_file_exists = os.path.isfile(os.path.join(zssgan_repo_path, 'legacy.py'))
        print(f"Verificación: Directorio 'dnnlib' existe en ZSSGAN: {dnnlib_dir_exists}")
        print(f"Verificación: Archivo 'legacy.py' existe en ZSSGAN: {legacy_file_exists}")
        if not dnnlib_dir_exists or not legacy_file_exists:
            print("ADVERTENCIA: 'dnnlib' o 'legacy.py' no encontrados en ZSSGAN.")

try:
    import dnnlib
    import legacy
    print("Módulos 'dnnlib' y 'legacy' importados correctamente.")
except ImportError as e_import:
    print(f"ERROR DE IMPORTACIÓN para dnnlib/legacy: {e_import}")


# Otros imports necesarios
import lpips
import clip
from insightface.app import FaceAnalysis # Necesita insightface instalado
print("Imports y configuración de rutas iniciales completados.")

Ruta 'c:\Users\juand\Documents\Universidad Septimo Semestre\Procesamiento Digital de Imagenes\Colab\ProyectoPDI\StyleGAN-nada/ZSSGAN' añadida a sys.path.
Módulos 'dnnlib' y 'legacy' importados correctamente.
Imports y configuración de rutas iniciales completados.


In [ ]:
local_project_data_path = os.path.join(current_working_directory, "ProjectFiles")
os.makedirs(local_project_data_path, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Dispositivo seleccionado: {device}")

# Modelo StyleGAN3 base pre-entrenado
base_stylegan3_url = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/stylegan3-t-ffhqu-1024x1024.pkl"
base_stylegan3_filename = "stylegan3-t-ffhqu-1024x1024.pkl"
base_stylegan3_path = os.path.join(local_project_data_path, base_stylegan3_filename)

# Nombre del archivo del generador StyleGAN3 especializado en Calima/Quimbaya
stylized_generator_filename = "G_stylized_CalimaQuimbaya.pkl"
stylized_generator_path = os.path.join(local_project_data_path, stylized_generator_filename)

#Directorio para las imagenes de entrada estilo
style_images_input_dir = os.path.join(local_project_data_path, "style_references_calima_quimbaya/")
os.makedirs(style_images_input_dir, exist_ok=True)
print(f"Directorio para imágenes de estilo (debes colocar tu imagen aquí): {style_images_input_dir}")

Dispositivo seleccionado: cuda
